In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd
import matplotlib as plt

In [ ]:
df = pd.read_csv('../input/tabular-playground-series-sep-2021/train.csv')
df.head(10)

In [ ]:
df.describe()

##  **We will start with handling missing values in the Dataset**

In [ ]:
df.isnull().sum()/len(df)

In [ ]:
fill_mean = lambda col : col.fillna(col.mean())
df = df.apply(fill_mean)

In [ ]:
df.isnull().sum()

* As you can see there are significant number of missing values that is around 2 % of the dataset. So, I replaced all missing values with **Mean**.

# Feature Selection
And the next step is about finding the relation between independent variables. As there are too many independent features plotting heat map could make whole visualization messy so I tried printing the **Correlation coefficient** and checked by running loop.

In [ ]:
df.corr()

In [ ]:
import scipy 
from scipy.stats.stats import pearsonr
for i in range(1,118):
    print(pearsonr(df['claim'], df['f'+ str(i)]))

In [ ]:
corr = df.corr()
criteria = corr[ corr.iloc[:]>= 0.1 ]     
print(len(criteria))


As you can see the count of Pearson Correlation Coefficient value greater tha 0.1 is 120, it indicates that no two independent features are significantly related so we cannot remove/drop any feature based on any **Correlation Coefficient**. 
So, now I have tried feature selection using **Mutual Information** in classification problem statement. 

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(df.drop(labels=['claim', 'id'], axis=1),
    df['claim'],
    test_size=0.3,
    random_state=0)

In [ ]:
from sklearn.feature_selection import mutual_info_classif
mutual_info = mutual_info_classif(X_train, y_train)
mutual_info

None of the feature have high mutual information with respect to dependent variable. So, I have tried checking for constant features using **Variance Threshold**. This could not be the effective way of removing features because the range of different independent features are very much different. So, I have just checked for constant values with variance threshold 0.

In [ ]:
from sklearn.feature_selection import VarianceThreshold
var_thres=VarianceThreshold(threshold=0)
var_thres.fit(df)

In [ ]:
var_thres.get_support()

As nothing worked I proceeded with all independent features into my training model.

In [ ]:
df['claim'].value_counts().sort_values(ascending=False).plot.bar(figsize=(10, 8))

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler= StandardScaler()
scaled_X_train = scaler.fit_transform(X_train)
scaled_X_test = scaler.fit(X_test)

In [ ]:
from sklearn.tree import DecisionTreeClassifier 
from sklearn.linear_model import LogisticRegression
clf = DecisionTreeClassifier()
clf.fit(scaled_X_train, y_train)

In [ ]:
Y_pred = clf.predict_proba(X_test)
print(Y_pred)

In [ ]:
from sklearn.metrics import roc_auc_score
print(roc_auc_score(y_test, Y_pred[:,0]))

In [ ]:
df1= pd.read_csv('../input/tabular-playground-series-sep-2021/test.csv')
print(df1.isnull().sum())

In [ ]:
fill_mean = lambda col : col.fillna(col.mean())
df1 = df1.apply(fill_mean)

In [ ]:
Test = np.array(df1.drop(['id'], axis = 1))
scaled_Test = scaler.fit(Test)

In [ ]:
Test_pred = clf.predict_proba(Test)
print(Test_pred)

In [ ]:
df3 = pd.DataFrame()
df3['id'] = df1['id']
df3['claim'] = Test_pred[:, 0]
df3.to_csv('./result.csv', index= False)